# Preparing protein structures for simulation: adding hydrogen atoms.

Protein structures that are solved by X-ray crystallography seldom include hydrogen atoms, but these must be added before the structures can be used for molecular simulation.

While in most cases the positions of "missing" hydrogens can be calculated unambiguously from a knowledge of heavy atom positions, there are some cases where this is not the case:

a) When alternative tautomeric states are possible. This is the case for histidine, which can exist as the delta- or epsilon tautomer:
<img src='images/Fig1.png' width=30%></td>

b) When hydrogens may be present or absent, depending on the pKa of the functional group and the (effective) pH of the environment. The most significant of these is again histidine, which with a pKa around 6.4 may be present in the protonated or neutral form (of which there are two as discussed above):

<img src='images/Fig2.png' width=20%>
But in rarer cases, other amino acids can also be found in protein structures in "unusual" protonation states. For example, ASP and GLU can sometimes be found in the protonated (neutral) form:
<img src='images/Fig3.png' width=45%>

And LYS can sometimes be found in the deprotonated neutral form too:
<img src='images/Fig4.png' width=20%>

Finally, CYS and TYR residues can be deprotonated is certain circumstances:
<img src='images/Fig5.png' width=40%>

Ensuring you run your simulation with all residues in the correct protonation state may be critical for getting correct behaviour. One way to tell your protein preparation workflow what is required is to modify residue names in a PDB format file, so:


    Neutral ASP: ASH
    Negative CYS: CYM
    Neutral GLU: GLH
    Neutral HIS: HIE or HSE (epsilon-protonated); HID or HSD (delta-protonated)
    Positive HIS: HIP or HSP
    Neutral LYS: LYN
    Negative TYR: TYM



In this tutorial you will see how you can use [PDB2PQR](http://server.poissonboltzmann.org/pdb2pqr) to help with this. `PDB2PQR` is avaialble via a web interface, but is also available as a Python software package that you can use from the command line (`pdb2pqr30`), and it's that version you will use here.

You will use `pdb2pqr30` to analyse the crystal structure of the cysteine protease cruzein (pdb code [2OZ2](https://www.rcsb.org/structure/2OZ2)). You will check what the software concludes about the protonation state of several important residues in this structure.

 
## Orientation:

This Jupyter notebook window contains the tutorial guide, and is where you will visualise the protein structure. To the right is a unix terminal window; this is where you will execute the commands to run the `pdb2pqr30` tool, and look at the output they produce. Be aware that this unix window is connected to the CCP-BioSim cloud service - you are not running on your own laptop/desktop.

## Part 1: visualisation of the protein.

In this cell we load the pdb file and visualize it using nglviewer:

In [ ]:
import nglview as nv
import mdtraj as mdt
import time
pdb_file = 'data/2oz2.pdb'
view = nv.show_file(pdb_file)
view.add_representation('ball+stick', 'acidic')
view.add_representation('ball+stick', 'basic')
view

The view shows most of the protein in ribbon form, but titratable sidechains arwe shown in full - as these are likely to be the most "interesting". Note that at the moment the structure contains no hydrogen atoms.

## Part 2: Hydrogen atom prediction using PDB2PQR

In the terminal window to the right, use `ls`,`cd` etc. to make sure you are in the `data` directory. You should be able to see the file `2oz2.pdb`.

type:

    pdb2pqr30 -h
    
You should see an extensive documentation of the program. 

We will use the options to combine the addition of hydrogen atoms with the analysis of probable ionisation states. Run `pdb2pqr30` as follows:

    pdb2pqr30 2oz2.pdb 2oz2.pqr --pdb-output 2oz2_pqr.pdb --ffout AMBER --titration-state-method propka
    
For a full explanation of what each of these arguments means, see the on-line documentation, but in brief we are telling the program to use the "propka" method to assign ionization states, and to produce an output PDB file in a format that could be recognised by the AMBER simulation package (so it will parameterise the amino acids correctly, based on any modified amino acid names).

A lot of info is written to the screen, and a new pdb format file `2oz2_pqr.pdb` is produced. The info written to the screen is probably only of interest to a specialist or software developer, the new pdb file contains everything you are interested in for now - so let's take a look.

In [ ]:
pdb_file2 = 'data/2oz2_pqr.pdb'
view2 = nv.show_file(pdb_file2)
view2.add_representation('ball+stick', 'ASH', color='orange')
view2.add_representation('ball+stick', 'ASP', color='red')
view2.add_representation('ball+stick', 'GLH', color='orange')
view2.add_representation('ball+stick', 'GLU', color='red')
view2.center('50:')
view2

Acidic residues in their "normal" deprotonated form (ASP, GLU) are shown in red; any "unusual" neutral acidic residues are shown in orange - are there any? If so, can you see why they may have been predicted to be in this unusual protonation state?

Now let's take a look at the histidines - which tautometic/ionization states have been predicted by PDB2PQR?

In [ ]:
view3 = nv.show_file(pdb_file2)
view3.add_representation('ball+stick', 'HID', color='cyan')
view3.add_representation('ball+stick', 'HIE', color='blue')
view3.add_representation('ball+stick', 'HIP', color='green')
view3

### Further study:

`pdb2pqr30` makes decisions about HIS tautomers based on which would produce the optimal H-bonding interactions with surrounding residues. Explore the environment around each HIS residue and see if you can convince yourself that `pdb2pqr30` has indeed done this.

---------

## Summary

The accurate modelling of hydrogen-bonding interactions, and perhaps even more so electrostatic interactions, is vital for meaningful simulations. So preparing proteins for simulation in the correct tautomeric and ionisation states is a very important part of the process.

Different molecular simulation packages will approach this in different ways - and not all are good. Use of a "third party" tool like `pdb2pqr30`, specifically designed to look at these issues, can be a big help.